In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-16 22:31:38--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   146MB/s    in 0.2s    

2024-12-16 22:31:39 (146 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code

In [5]:
#Merge df_books and df_ratings
df = df_ratings.merge(df_books, on='isbn',how="left")


In [6]:
df.head

<bound method NDFrame.head of            user         isbn  rating  \
0        276725   034545104X     0.0   
1        276726   0155061224     5.0   
2        276727   0446520802     0.0   
3        276729   052165615X     3.0   
4        276729   0521795028     6.0   
...         ...          ...     ...   
1149775  276704   1563526298     9.0   
1149776  276706   0679447156     0.0   
1149777  276709   0515107662    10.0   
1149778  276721   0590442449    10.0   
1149779  276723  05162443314     8.0   

                                                     title             author  
0                                     Flesh Tones: A Novel         M. J. Rose  
1                                         Rites of Passage         Judith Rae  
2                                             The Notebook    Nicholas Sparks  
3                                           Help!: Level 1      Philip Prowse  
4        The Amsterdam Connection : Level 4 (Cambridge ...        Sue Leather  
...                                                    ...                ...  
1149775  Get Clark Smart : The Ultimate Guide for the S...       Clark Howard  
1149776  Eight Weeks to Optimum Health: A Proven Progra...        Andrew Weil  
1149777   The Sherbrooke Bride (Bride Trilogy (Paperback))  Catherine Coulter  
1149778                                  Fourth Grade Rats     Jerry Spinelli  
1149779                                                NaN                NaN  

[1149780 rows x 5 columns]>

In [7]:
df.shape

(1149780, 5)

In [8]:
#Get users who left more than 200 reviews
#Get books with  more than 100
users= df["user"].value_counts()
isbn= df["isbn"].value_counts()

users = users[users >= 200].index
isbn = isbn[isbn >= 100].index



In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()